In [4]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [5]:
cont_info = {
            'Asia':['59af92b13e70001c1bd78e53', 31]}

In [6]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as bs
from geopy.geocoders import Nominatim

# Function to get latitude and longitude from city name
def get_coordinates(city_name):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Initialize an empty DataFrame to store all scraped data
df_all = pd.DataFrame()

# Loop through each continent in your cont_info dictionary
for key, val in cont_info.items():
    cont_name = key
    cont_url = val[0]
    cont_pgs = val[1]
    
    dfs_continent = []  # List to store DataFrames for each continent
    
    # Loop through each page of data for the current continent
    for pg_num in range(1, cont_pgs + 1):
        url = f'https://www.iqair.com/world-most-polluted-cities?continent={cont_url}&country=&state=&page={pg_num}&perPage=50&cities='
        page = req.get(url)
        soup = bs(page.content, 'html.parser')
        
        # Find the table containing data
        tbl = soup.find('table')
        
        if tbl:
            # Extract column names
            col_names = [col.text.strip() for col in tbl.find_all('th')]
            
            # Extract row data
            rows = tbl.find_all('tr')
            row_dat = {}
            for row in rows:
                dat = row.find_all('td')
                dat_list = [d.text.strip() for d in dat]
                if len(dat_list) > 0:
                    dat_key = int(dat_list[0])
                    dat_val = dat_list[1:]
                    row_dat[dat_key] = dat_val
            
            # Convert row data to DataFrame
            df = pd.DataFrame.from_dict(row_dat, orient='index')
            df.columns = col_names[1:]
            
            # Append the DataFrame to the list for this continent
            dfs_continent.append(df)
    
    # Concatenate all DataFrames for this continent into one DataFrame
    if dfs_continent:
        df_continent = pd.concat(dfs_continent)
        
        # Add a column for the continent name
        df_continent['continent'] = cont_name
        
        # Append the continent DataFrame to the overall DataFrame
        df_all = pd.concat([df_all, df_continent])

# Reset index of the final DataFrame
df_all.reset_index(drop=True, inplace=True)

# Add latitude and longitude columns based on city names
df_all['Latitude'], df_all['Longitude'] = zip(*df_all['City'].apply(get_coordinates))
file_path = #/Your file path # Replace 'path_to_your_file.xlsx' with your desired file path

# Export df_all to an Excel file
df_all.to_csv(file_path, index=False)
# Now df_all contains all the scraped data for all continents with latitude and longitude columns
